## Import required packages

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dateutil

## Load necessary datasets in form of pandas dataframe

In [2]:
## load aggregate measure data

# meeting minutes
df_measure_mm = pd.read_excel("../data/market_analysis_data/aggregate_measure_mm.xlsx", usecols=["ReleaseDate", "EndDate", "our_measure"])
df_measure_mm["EndDate"] = pd.to_datetime(df_measure_mm["EndDate"], format='%B/%d/%Y')
df_measure_mm["delay"] = (df_measure_mm["ReleaseDate"] - df_measure_mm["EndDate"]).dt.days

# speeches
df_measure_sp = pd.read_excel("../data/market_analysis_data/aggregate_measure_sp.xlsx", usecols=["Date", "our_measure"])
df_measure_sp['Date'] = df_measure_sp['Date'].apply(dateutil.parser.parse)
df_measure_sp["Date"] = pd.to_datetime(df_measure_sp["Date"], format='%m/%d/%Y')

# press conferences 
df_measure_pc = pd.read_excel("../data/market_analysis_data/aggregate_measure_pc.xlsx", usecols=["EndDate", "our_measure"])
df_measure_pc["EndDate"] = pd.to_datetime(df_measure_pc["EndDate"], format='%B/%d/%Y')


## load US Treasury Data 
# Sorce: https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value=all

df_treasury = pd.read_csv("../data//market_analysis_data/daily-treasury-rates.csv", usecols=["Date", "1 Yr", "10 Yr", "3 Mo"])
df_treasury['Date'] = df_treasury['Date'].astype('datetime64[ns]')
df_treasury['slope_10_1'] = df_treasury['10 Yr'] - df_treasury['1 Yr']
df_treasury['slope_10y_3m'] = df_treasury['10 Yr'] - df_treasury['3 Mo']
df_treasury = df_treasury.iloc[::-1]

## Treasury Yield Analysis

In [35]:
######################## Treasury Yield Analysis ########################
df_merge = pd.merge(df_measure_pc, df_treasury, left_on="EndDate", right_on="Date", how="left") 
# (df_measure_mm, "ReleaseDate"), (df_measure_sp, "Date"), (df_measure_pc, "EndDate")
df_merge = df_merge.dropna()

# run OLS
var_x = df_merge["our_measure"].tolist()
var_x = sm.add_constant(var_x)

var_y = df_merge["1 Yr"].tolist() #"1 Yr", "10 Yr", "slope_10_1", slope_10y_3m, 3 Mo

result = sm.OLS(var_y, var_x).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.206
Model:                            OLS   Adj. R-squared:                  0.193
Method:                 Least Squares   F-statistic:                     15.60
Date:                Fri, 23 Dec 2022   Prob (F-statistic):           0.000209
Time:                        18:00:29   Log-Likelihood:                -80.487
No. Observations:                  62   AIC:                             165.0
Df Residuals:                      60   BIC:                             169.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9842      0.116      8.469      0.0

## Get and store residual from Treasury regression

In [36]:
### get residual from regression
pred_y = result.predict(var_x)
residual_list = [(var_y[i]-pred_y[i]) for i in range(len(var_x)) ]

df_merge["residual_1yr"] = residual_list

df_measure_residual = df_merge[["EndDate", "residual_1yr"]]
#print(df_measure_residual.tail())
df_measure_residual = df_measure_residual.dropna()
#print(df_measure_residual.shape)
df_measure_residual.to_excel("../data/market_analysis_data/residual_measure_pc.xlsx", index=False)